# Import modules
CDC data set description link [here](https://wwwn.cdc.gov/nchs/nhanes/search/datapage.aspx?Component=Questionnaire&CycleBeginYear=2015)

In [1]:
import pdb
import glob

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.feature_selection import VarianceThreshold, SelectFromModel, SelectKBest, mutual_info_classif, mutual_info_regression

import nhanes as nhanes

%matplotlib notebook

## Settings

In [2]:
DATA_PATH = 'C:/Users/allen/Documents/Git-Repos/Opportunistic/CDC/NHANES/'
DATASET = 'cancer'

### Note: 
The code below loads each dataset: dataset_features, dataset_targets

Here, all datasets are defined explicitly (see nhanes.py).

In [3]:
ds = nhanes.Dataset(DATA_PATH)
ds.load_cancer()
n_fe = ds.features.shape[1]
n_classes = 2

Processing: Questionnaire\RHQ_D.XPT                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                     

Processing: Examination\AUXTYM_C.XPT                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    

Processing: Laboratory\LAB10.XPT                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                        

Processing: Questionnaire\WHQ_H.XPT                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                     

Extract the features and targets.

In [4]:
# indx = np.argwhere(ds.targets != 3)
dataset_features = ds.features # [ds.targets != 3]
dataset_targets = ds.targets # [ds.targets != 3]

# Pre-fix
# dataset_features = ds.features
# dataset_targets = ds.targets

## Preprocessing of Data
### Drop features with too-low variance

In [20]:
dataset_features_sel = dataset_features.copy()

var_thresh = .001
dropped_keys = []
for key, value in dataset_features_sel.iteritems():
    if(value.var() < var_thresh):
        dataset_features_sel.drop(key, axis=1, inplace=True)
        dropped_keys.append(key)

print("Dropped %i keys, they were:\n %s" %(len(dropped_keys), dropped_keys))

Dropped 26 keys, they were:
 ['BPXCHR', 'MCQ080#7.0', 'MCQ080#9.0', 'MCQ365A#7.0', 'MCQ365A#9.0', 'MCQ365B#7.0', 'MCQ365B#9.0', 'MCQ365D#7.0', 'MCQ365D#9.0', 'MCQ365C#7.0', 'MCQ365C#9.0', 'MCQ370A#7.0', 'MCQ370A#9.0', 'MCQ370B#7.0', 'MCQ370B#9.0', 'MCQ370C#7.0', 'MCQ370C#9.0', 'MCQ370D#7.0', 'MCQ370D#9.0', 'PAQ677', 'SLQ050#7.0', 'SLQ050#9.0', 'SMQ621', 'MCQ160J#9.0', 'SMQ020#7.0', 'SMQ020#9.0']


### Calculate mutual information

In [ ]:
mutual_info = mutual_info_regression(dataset_features_sel, dataset_targets)
# print(mutual_info)

### Dropping features with low mutual information
    __Conditioned on target variable__

In [ ]:
mi_df = pd.DataFrame(mutual_info)
mi_df = mi_df.transpose()
selector = mi_df > .01
# print(mi_df)

to_drop = []
for row in selector.itertuples():
    for i in range(len(row)-1):
        if((row[i])):
            to_drop.append(i)

mu_features_sel = pd.DataFrame(dataset_features_sel.copy())
mu_features_sel = mu_features_sel.drop(columns=to_drop)
print(mu_features_sel.shape())

## Train/Test Separation

In [ ]:
perm = np.random.permutation(dataset_targets.shape[0])
dataset_features = dataset_features_sel[perm]
dataset_targets = dataset_targets[perm]

print("dataset_features Shape: %s, dataset_targets Shape: %s" % (dataset_features.shape, dataset_targets.shape))

def get_batch(n_size, phase):
    # select indices
    n_samples = dataset_features.shape[0]
    n_classes = int(dataset_targets.max() + 1)
    if phase == 'test':
        inds_sel = np.arange(0, int(n_samples*0.15), 1)
    elif phase == 'validation':
        n_samples = dataset_features.shape[0]
        inds_sel = np.arange(int(n_samples*0.15), int(n_samples*0.30), 1)
    elif phase == 'train':
        n_samples = dataset_features.shape[0]
        inds_sel = np.arange(int(n_samples*0.30), n_samples, 1)
    else:
        raise NotImplementedError
    inds_sel = np.random.permutation(inds_sel)
    batch_inds = []
    for cl in range(n_classes):
        inds_cl = inds_sel[dataset_targets[inds_sel] == cl]
        batch_inds.extend(inds_cl[:n_size//n_classes])
    batch_inds = np.random.permutation(batch_inds)
    
    return dataset_features[batch_inds], dataset_targets[batch_inds]
    
features_trn, targets_trn = get_batch(n_size=5000, phase='train')
features_tst, targets_tst = get_batch(n_size=1000, phase='test')

## Classification

In [ ]:
clf = RandomForestClassifier(n_estimators=100)
clf.fit(features_trn, targets_trn)
preds_tst = clf.predict(features_tst)
accu = np.mean(preds_tst==targets_tst)
print('accu_tst_RFC', accu)

clf = SVC(gamma='auto')
clf.fit(features_trn, targets_trn)
preds_tst = clf.predict(features_tst)
accu = np.mean(preds_tst==targets_tst)
print('accu_tst_SVC', accu)

clf = LogisticRegression(solver='lbfgs', max_iter=200)
clf.fit(features_trn, targets_trn)
preds_tst = clf.predict(features_tst)
accu = np.mean(preds_tst==targets_tst)
print('accu_tst_LR', accu)


In [ ]:
print(classification_report(targets_tst, preds_tst))

### Accuracies from baseline: 
#### Cancer (ds.load_cancer()):
* accu_tst_RFC 0.758
* accu_tst_SVC 0.759
* accu_tst_LR 0.768

#### Arthiritis (ds.load_arthiritis()):
* accu_tst_RFC 0.753
* accu_tst_SVC 0.754
* accu_tst_LR 0.773